In [8]:
import requests
import geojson
import aiohttp
import asyncio
from geocodio import GeocodioClient
from app.backend.app.externalAPI import GeocodioRequests

import pandas as pd
import matplotlib as plt
import rasterio
import numpy as np

In [ ]:
# with rasterio.open("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif") as src:
#     data = src.read()
#     image = src.read(1)
#     clusters = src.read(2)
#     bounds = src.bounds

In [182]:
import rasterio
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import cascaded_union
from rasterio import Affine

def get_polygons_center(clustered_tif_path):
    with rasterio.open(clustered_tif_path) as src:
        data = src.read()
        image = src.read(1)
        global clusters
        clusters = src.read(2)

    class_vals = set(clusters.flatten()) - {-1}
    centers = []
    pcords = []
    manual_centers = []
    for class_val in class_vals:
        mask = clusters == class_val
        shapes = rasterio.features.shapes(mask.astype('uint8'), transform=src.transform)
        local_mc = []
        polygons = []
        for polygon, value in shapes:
            polygons.append(Polygon(polygon['coordinates'][0]))
            mc = [0,0]
            mc[1] = sum([i[0] for i in polygon['coordinates'][0]])/len(polygon['coordinates'][0])
            mc[0] = sum([i[1] for i in polygon['coordinates'][0]])/len(polygon['coordinates'][0])
            local_mc.append(tuple(mc))
            pcords.append(polygon['coordinates'][0])
        # print(list(zip(local_mc)))
        manual_centers.append(tuple(sum(coords)/len(coords) for coords in zip(*local_mc)))
        multipolygon = MultiPolygon(polygons)
        center_pixel = multipolygon.centroid
        center_realworld = rasterio.transform.xy(src.transform, center_pixel.x, center_pixel.y)
        centers.append((center_realworld[1], center_realworld[0]))
    print(manual_centers)
    print(min(manual_centers))
    print(max(manual_centers))
    print(len(manual_centers))
    return manual_centers


In [139]:
centers = get_polygons_center("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif")
len(centers)

[(41.08957727110209, -73.75602149612448), (40.991029390839955, -74.2540645717108), (41.24488820996321, -74.19354640405001), (41.12252527629722, -74.26162003243213), (41.35276198372513, -73.67408741594161), (41.29823589213861, -73.79751541748706), (41.211320988409035, -73.8387896503878), (40.84061047624367, -73.748040330874), (41.21617132778225, -73.84483251069511), (41.24890080356474, -73.73587480827594), (40.77771044019959, -73.97491330088364), (41.23832088915092, -74.10733565544045), (41.02896761282016, -73.99403092781954), (40.98307355409337, -74.01468779448847), (41.33687099868878, -73.98111275723089), (40.88197420832995, -73.77747709379886), (40.820704856686106, -73.95792292884616), (40.74882374454802, -74.11341539880898), (41.12609218668781, -73.96887873773433), (40.64142960965954, -74.18513147035875), (40.68271140881101, -73.86940754744334), (40.77180179023123, -73.94049915960714), (40.77410080115263, -73.93598446574204), (40.760602315482735, -73.93607625987912), (40.75262763903

247

In [140]:
min(centers)

(40.4233495399713, -74.17441381166898)

In [141]:
max(centers)

(41.38895668460448, -73.68139153365186)

In [142]:
gr = GeocodioRequests()
address_sets = gr.client.reverse(centers, fields = ["acs-economics"])

In [156]:
address_sets[0]

{'results': [{'address_components': {'number': '32',
    'street': 'State Rte 22',
    'formatted_street': 'State Rte 22',
    'city': 'Valhalla',
    'county': 'Westchester County',
    'state': 'NY',
    'zip': '10595',
    'country': 'US'},
   'formatted_address': '32 State Rte 22, Valhalla, NY 10595',
   'location': {'lat': 41.089896, 'lng': -73.754399},
   'accuracy': 0.99,
   'accuracy_type': 'rooftop',
   'source': 'Westchester',
   'fields': {'census': {'2021': {'census_year': 2021,
      'state_fips': '36',
      'county_fips': '36119',
      'tract_code': '012301',
      'block_code': '2000',
      'block_group': '2',
      'full_fips': '361190123012000',
      'place': None,
      'metro_micro_statistical_area': {'name': 'New York-Newark-Jersey City, NY-NJ-PA',
       'area_code': '35620',
       'type': 'metropolitan'},
      'combined_statistical_area': {'name': 'New York-Newark, NY-NJ-CT-PA',
       'area_code': '408'},
      'metropolitan_division': {'name': 'New York-Je

In [157]:
household_income = address_sets[2]['results'][0]['fields']['acs']['economics']['Median household income']['Total']['value']
household_income

{'meta': {'table_id': 'B19013', 'universe': 'Households'},
 'Total': {'value': 91250, 'margin_of_error': 49812}}

In [160]:
print([address_sets[0]['results'][i]['location']['lat'] for i in range(len(address_sets[0]['results']))])
print([address_sets[0]['results'][i]['location']['lng'] for i in range(len(address_sets[0]['results']))])
print(centers[0])

[41.089896, 41.097692, 41.097487, 41.09731, 41.098093, 41.098093, 41.095577, 41.094194, 41.092902]
[-73.754399, -73.760441, -73.761207, -73.762283, -73.760421, -73.760421, -73.765901, -73.767164, -73.768307]
(41.08957727110209, -73.75602149612448)


In [167]:
address_sets[38]

{'results': [{'address_components': {'number': '123',
    'street': 'Varick',
    'suffix': 'Ave',
    'formatted_street': 'Varick Ave',
    'city': 'Brooklyn',
    'county': 'Kings County',
    'state': 'NY',
    'zip': '11237',
    'country': 'US'},
   'formatted_address': '123 Varick Ave, Brooklyn, NY 11237',
   'location': {'lat': 40.709914, 'lng': -73.928135},
   'accuracy': 0.99,
   'accuracy_type': 'rooftop',
   'source': 'City of New York',
   'fields': {'census': {'2021': {'census_year': 2021,
      'state_fips': '36',
      'county_fips': '36047',
      'tract_code': '044902',
      'block_code': '3005',
      'block_group': '3',
      'full_fips': '360470449023005',
      'place': {'name': 'New York', 'fips': '3651000'},
      'metro_micro_statistical_area': {'name': 'New York-Newark-Jersey City, NY-NJ-PA',
       'area_code': '35620',
       'type': 'metropolitan'},
      'combined_statistical_area': {'name': 'New York-Newark, NY-NJ-CT-PA',
       'area_code': '408'},
     

In [194]:
cords = []
for result in address_sets:
    if result['results']:
        # check is for keyerror due to bad data.(value if exists else 0)
        income_list = [result['results'][i]['fields']['acs']['economics']['Median household income']['Total']['value'] 
                       if result['results'][i]['fields']['acs']['economics']['Median household income'].get('Total', 0) 
                       else 0 for i in range(len(result['results']))]
        income = sum(list(filter(lambda x: x != 0, income_list)))/len(income_list)
        cords.append(income)

In [195]:
cords

[146636.77777777778,
 167841.0,
 91250.0,
 128688.0,
 119538.23076923077,
 152981.0,
 202500.0,
 161670.33333333334,
 240500.8,
 168012.66666666666,
 136507.0,
 99308.0,
 194266.0,
 135385.0,
 77829.0,
 58906.25,
 25738.0,
 130666.0,
 190781.0,
 84952.45454545454,
 74496.0,
 78243.5,
 0.0,
 23038.0,
 53701.0,
 0.0,
 53701.0,
 0.0,
 0.0,
 72566.0,
 85833.0,
 0.0,
 41968.666666666664,
 87788.0,
 116932.0,
 77660.8,
 0.0,
 0.0,
 3431.923076923077,
 0.0,
 67798.0,
 47218.0,
 85841.0,
 92315.0,
 128571.0,
 0.0,
 66674.75,
 0.0,
 136214.0,
 77038.0,
 114063.0,
 23319.0,
 47569.0,
 45378.0,
 41537.666666666664,
 250001.0,
 36552.0,
 47019.0,
 77072.0,
 93611.0,
 53900.0,
 173750.0,
 46915.0,
 46577.53846153846,
 114076.0,
 54743.0,
 96771.0,
 66250.0,
 87604.0,
 116435.0,
 211250.0,
 144031.0,
 74080.0,
 76139.0,
 136237.6,
 103781.0,
 104524.0,
 0.0,
 186250.0,
 250001.0,
 250001.0,
 142213.5,
 96875.0,
 96875.0,
 174659.0,
 163517.0,
 125278.0,
 250001.0,
 213889.0,
 141172.88888888888,
 82

In [203]:
cluster_score = {c: 0 for c in np.unique(clusters)}
for i in range(0, len(cluster_score)-1):
    cluster_score[i] = int(cords[i])
cluster_score

{-1: 0,
 0: 146636,
 1: 167841,
 2: 91250,
 3: 128688,
 4: 119538,
 5: 152981,
 6: 202500,
 7: 161670,
 8: 240500,
 9: 168012,
 10: 136507,
 11: 99308,
 12: 194266,
 13: 135385,
 14: 77829,
 15: 58906,
 16: 25738,
 17: 130666,
 18: 190781,
 19: 84952,
 20: 74496,
 21: 78243,
 22: 0,
 23: 23038,
 24: 53701,
 25: 0,
 26: 53701,
 27: 0,
 28: 0,
 29: 72566,
 30: 85833,
 31: 0,
 32: 41968,
 33: 87788,
 34: 116932,
 35: 77660,
 36: 0,
 37: 0,
 38: 3431,
 39: 0,
 40: 67798,
 41: 47218,
 42: 85841,
 43: 92315,
 44: 128571,
 45: 0,
 46: 66674,
 47: 0,
 48: 136214,
 49: 77038,
 50: 114063,
 51: 23319,
 52: 47569,
 53: 45378,
 54: 41537,
 55: 250001,
 56: 36552,
 57: 47019,
 58: 77072,
 59: 93611,
 60: 53900,
 61: 173750,
 62: 46915,
 63: 46577,
 64: 114076,
 65: 54743,
 66: 96771,
 67: 66250,
 68: 87604,
 69: 116435,
 70: 211250,
 71: 144031,
 72: 74080,
 73: 76139,
 74: 136237,
 75: 103781,
 76: 104524,
 77: 0,
 78: 186250,
 79: 250001,
 80: 250001,
 81: 142213,
 82: 96875,
 83: 96875,
 84: 174

In [211]:
df = pd.DataFrame(list(cluster_score.items()), columns=["cluster", "income"])
df.to_csv('average_income.csv', index=False)
df

,cluster,income
0,-1,0
1,0,146636
2,1,167841
3,2,91250
4,3,128688
...,...,...
243,242,178750
244,243,91250
245,244,98036
246,245,139865
